# Simple IEX Stock Fetcher

The IEX free message quota only gets one year of data for 200 stocks, so run this, fetcher stock data and save to disk every month.


## Test api

In [1]:
import iex
import key
iex.init(key.test_token, api='sandbox')
aapl = iex.Stock('AAPL')
aapl.chart('5d', chartCloseOnly=True)

,close,volume,change,changePercent,changeOverTime
date,,,,,
2019-12-02,275.94,23955391,0.00,0.0000,0.000000
2019-12-03,267.31,29436533,-4.71,-1.8220,-0.018040
2019-12-04,270.50,17105848,2.30,0.9027,-0.009595
2019-12-05,271.09,19544976,3.90,1.5050,0.005488
2019-12-06,270.78,27588546,5.23,1.9732,0.025005


In [11]:
aapl.dividends('1y')

,paymentDate,recordDate,declaredDate,amount,flag,currency,description,frequency,date
exDate,,,,,,,,,
2019-11-22,2019-11-28,2019-11-21,2019-11-12,0.78,ahsC,USD,ayriahsrO eSdnr,lueraryqt,2019-12-07
2019-08-16,2019-08-20,2019-08-20,2019-08-02,0.79,sCha,USD,Ohey drainrsrSa,aerqurtly,2019-12-07
2019-05-22,2019-05-29,2019-05-18,2019-05-12,0.77,hCas,USD,nOrSsyairrhea d,layrertqu,2019-12-07
2019-02-22,2019-02-15,2019-02-12,2019-01-30,0.73,Chas,USD,rsdai yrneSOhra,eqratlyur,2019-12-07


In [14]:
# iex.init(key.token, api='cloud')
# aapl.dividends('1y')

,paymentDate,recordDate,declaredDate,amount,flag,currency,description,frequency,date
exDate,,,,,,,,,
2019-11-07,2019-11-14,2019-11-11,2019-10-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-08-09,2019-08-15,2019-08-12,2019-07-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-05-10,2019-05-16,2019-05-13,2019-04-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-02-08,2019-02-14,2019-02-11,2019-01-29,0.73,Cash,USD,Ordinary Shares,quarterly,2019-12-07


# Fetch Symobols

In [10]:
iex.init(key.token, api='cloud')
symbols = iex.Reference.symbols()
symbols.to_csv("./symbols.csv")
iex.init(key.test_token, api='sandbox')

type	refers to the common issue type

    ad - ADR
    re - REIT
    ce - Closed end fund
    si - Secondary Issue
    lp - Limited Partnerships
    cs - Common Stock
    et - ETF
    wt - Warrant
    oef - Open Ended Fund
    cef - Closed Ended Fund
    ps - Preferred Stock
    ut - Unit
    struct - Structured Product

In [21]:
symbols.groupby('type').symbol.count()

type
ad         402
cef        506
cs        4536
et        2339
ps         762
rt          23
struct       2
temp         1
ut         116
wt         177
Name: symbol, dtype: int64

# Fetcher Functions

In [2]:
import os, sys
import pandas as pd
chart_range = [(5,'5d'), (20,'1m'), (75,'3m'), (165,'6m'), (341,'1y'), (715,'2y'), (99999, '5y')]

def range_since_last(df):
    days = pd.Timedelta(pd.datetime.now() - df.index[-1]).days
    bdays = len(pd.bdate_range(df.index[-1], pd.datetime.now())) - 1
    if bdays <= 0:
        return None
    for range_, param in chart_range:
        if days < range_:
            return param
    return '5y'

def fetch_on_demand(filename, index_col, fetch_call):
    if os.path.exists(filename):
        df = pd.read_csv(filename, index_col=index_col, parse_dates=True)
        param = range_since_last(df)
        sys.stdout.write(str(param) + "      ")
        if param is None:
            return
        # fetcher missing period
        try:
            df_append = fetch_call(param)
        except EOFError:  # empty data
            return
        df = pd.concat([df, df_append[~df_append.index.isin(df.index)]])
        df.sort_index(inplace=True)
        df.to_csv(filename)
    else:
        sys.stdout.write('new 5y')
        try:
            df = fetch_call('5y')
        except EOFError:  # empty data
            return
        df.sort_index(inplace=True)
        df.to_csv(filename)

## Fetch Adjustment

In [3]:
import numpy as np
symbols = pd.read_csv('./symbols.csv', index_col='date', parse_dates=True)
symbols = symbols[(symbols.type == 'ad') | (symbols.type == 'cs')]
symbols = symbols.symbol.values
symbols = np.append(symbols, ['SPY', 'QQQ'])
symbols, symbols.shape

(array(['A', 'AA', 'AACG', ..., 'ZYXI', 'SPY', 'QQQ'], dtype=object), (4940,))

In [16]:
# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

for k in symbols:
    sys.stdout.write('\r{}   '.format(k))
    filename = "./adjustments/{}.csv".format(k)
    fetch_on_demand(filename, 'exDate', iex.Stock(k).dividends)
print('ok')

AAMC   new 5y  ok


## Fetch stocks

In [10]:
import requests
import pandas as pd
import time

In [6]:
etfs = pd.read_html(requests.get('https://etfdailynews.com/etf/spy/', headers={'User-agent': 'Mozilla/5.0'}).text,
             attrs={'id': 'etfs-that-own'})
spy = [x for x in etfs[0].Symbol.values.tolist() if isinstance(x, str)]

In [23]:
correction = {'PCLN':'BKNG', 'DWDP':'DD', 'CELG':'skip..', 'TWX':'skip..'} # 'CELG':'CELGRT'

In [24]:
# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

# save historical price to disk
for k in spy:
    k = correction[k] if k in correction else k
    if k =='skip..': continue
    sys.stdout.write('\r{}   '.format(k))
    filename = "./daily/{}.csv".format(k)
    fetch_on_demand
    
    if os.path.exists(filename):
        # fetcher missing period
        df = pd.read_csv(filename, index_col='date', parse_dates=True)
        days = pd.Timedelta(pd.datetime.now() - df.index[-1]).days
        bdays = len(pd.bdate_range(df.index[-1], pd.datetime.now())) - 1
        for day, param in chart_range:
            if bdays <= 0:
                break
            elif days < day:
                print(param)
                df_append = iex.Stock(k).chart(param)
                pd.concat([df, df_append], sort=True).to_csv(filename)
                break
    else:
        print('full')
        iex.Stock(k).chart('5y').to_csv(filename)
    


UPS   full
NEE   full
AXP   full
CHTR   full
CVS   full
BLK   full


Exception: GET https://cloud.iexapis.com/stable/stock/BLK/chart/max?token=pk_7cc910b360ea4a31813e0ca78a02243b 402 You have exceeded your allotted message quota. Please enable pay-as-you-go to regain access

In [3]:
df = aapl.chart('5d')

In [5]:
df.index.dtype

dtype('<M8[ns]')

In [14]:
isinstance(df.index, pd.DatetimeIndex)

True

In [13]:
df.index

DatetimeIndex(['2019-10-21', '2019-10-22', '2019-10-23', '2019-10-24',
               '2019-10-25'],
              dtype='datetime64[ns]', name='date', freq=None)